In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# creating the input array

inputs = np.array([
    [25, 60, 10],
    [28, 55, 15],
    [30, 50, 20],
    [32, 48, 18],
    [26, 62, 12],
    [24, 58, 8],
    [29, 54, 22],
    [31, 53, 25],
    [27, 61, 14],
    [33, 49, 17],
    [22, 65, 9],
    [23, 63, 11],
    [30, 52, 19],
    [28, 56, 16],
    [25, 59, 13]
], dtype = 'float32')

# creating the target array

target = np.array([
    [30, 25],
    [32, 22],
    [35, 20],
    [31, 23],
    [28, 26],
    [29, 24],
    [33, 18],
    [30, 21],
    [34, 19],
    [31, 22],
    [27, 28],
    [28, 27],
    [36, 16],
    [33, 20],
    [29, 25]
], dtype = 'float32')


In [3]:
inputs = torch.from_numpy(inputs)
target = torch.from_numpy(target)

In [4]:
# create a dataloader 

from torch.utils.data import  TensorDataset

In [5]:
train_ds = TensorDataset(inputs, target)

train_ds[0:3]

(tensor([[25., 60., 10.],
         [28., 55., 15.],
         [30., 50., 20.]]),
 tensor([[30., 25.],
         [32., 22.],
         [35., 20.]]))

In [6]:
# create the dataloader for splitting the data into
# batches


from torch.utils.data import  DataLoader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

In [7]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[33., 49., 17.],
        [28., 55., 15.],
        [31., 53., 25.],
        [22., 65.,  9.],
        [30., 52., 19.]])
tensor([[31., 22.],
        [32., 22.],
        [30., 21.],
        [27., 28.],
        [36., 16.]])


In [8]:
# defining the model

model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0808, -0.1810,  0.2548],
        [ 0.4591, -0.3780, -0.0868]], requires_grad=True)
Parameter containing:
tensor([0.2322, 0.4290], requires_grad=True)


In [9]:
# parameters method returns all the weight and bias matrices present in
# the model


list(model.parameters())

[Parameter containing:
 tensor([[-0.0808, -0.1810,  0.2548],
         [ 0.4591, -0.3780, -0.0868]], requires_grad=True),
 Parameter containing:
 tensor([0.2322, 0.4290], requires_grad=True)]

In [10]:
# generate predictions

preds = model(inputs)
preds

tensor([[-10.0994, -11.6396],
        [ -8.1628,  -8.8064],
        [ -6.1453,  -6.4324],
        [ -6.4547,  -4.5845],
        [-10.0326, -12.1101],
        [-10.1663, -11.1692],
        [ -6.2788,  -8.5771],
        [ -5.4950,  -7.5413],
        [ -9.4227, -11.4466],
        [ -6.9713,  -4.4165],
        [-11.0167, -14.8202],
        [-10.2259, -13.7787],
        [ -6.7622,  -7.1015],
        [ -8.0890,  -9.2712],
        [ -9.1539, -11.5221]], grad_fn=<AddmmBackward0>)

In [12]:
# computing the loss

def loss_fn(y_pred, y_true):
    diff = y_pred - y_true
    return torch.sum(diff * diff)/diff.numel()

In [14]:
loss = loss_fn(model(inputs), target)
print(loss)

tensor(1303.7799, grad_fn=<DivBackward0>)


In [15]:
# define the optimizer 

opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [20]:
# utility function to train the model

def fit(num_epochs, model, loss_fn, opt, train_dl):
    
     # repeat the given number of epochs
     for epoch in range(num_epochs):
         # train with batches of data
         for xb, yb in train_dl:
             
             #1. generate predictions
             pred = model(xb)
             
             # 2. calculate the loss
             loss = loss_fn(pred, yb)
             
             #3. get the gradient of the loss w.r.t parameters (weights & bias)
             loss.backward()
             
             # 4. optimize the parameters by using the gradiets
             opt.step()
             
             #5. Reset the gradient to zero
             opt.zero_grad()
         if (epoch + 1) % 10 == 0:
            print(f'Epoch : {epoch+1}, number of epochs:{num_epochs}, Loss : {loss.item()}')
            
            
             
        

In [22]:
#training the model for 100 epoch

fit(100, model, loss_fn, opt, train_dl)

Epoch : 10, loss:100, Loss : 115.71683502197266
Epoch : 20, loss:100, Loss : 23.209171295166016
Epoch : 30, loss:100, Loss : 7.619116306304932
Epoch : 40, loss:100, Loss : 18.529483795166016
Epoch : 50, loss:100, Loss : 22.701805114746094
Epoch : 60, loss:100, Loss : 14.656954765319824
Epoch : 70, loss:100, Loss : 22.23255157470703
Epoch : 80, loss:100, Loss : 10.626954078674316
Epoch : 90, loss:100, Loss : 8.142248153686523
Epoch : 100, loss:100, Loss : 15.619306564331055


In [23]:
preds = model(inputs)

preds

tensor([[29.6159, 21.4220],
        [30.5242, 22.6662],
        [31.2395, 23.2871],
        [30.1285, 24.4033],
        [31.3058, 22.1755],
        [27.9260, 20.6686],
        [33.2261, 22.9844],
        [34.4922, 24.0457],
        [31.9718, 22.6436],
        [30.2556, 25.1515],
        [30.3363, 20.0580],
        [30.6610, 20.4310],
        [31.5149, 23.5073],
        [31.2727, 22.7313],
        [30.4961, 21.2370]], grad_fn=<AddmmBackward0>)

In [24]:
target

tensor([[30., 25.],
        [32., 22.],
        [35., 20.],
        [31., 23.],
        [28., 26.],
        [29., 24.],
        [33., 18.],
        [30., 21.],
        [34., 19.],
        [31., 22.],
        [27., 28.],
        [28., 27.],
        [36., 16.],
        [33., 20.],
        [29., 25.]])

In [ ]:
# AND that is the basic implementation of linear regression in NN


